# Acoustics

In [1]:
%matplotlib inline

In [2]:
%config InlineBackend.figure_format = 'svg'
import numpy as np
from exact_solvers import acoustics, acoustics_demos
from IPython.display import IFrame

In this chapter we consider our first *system* of hyperbolic conservation laws.  We study the acoustics equations that were introduced briefly in [Introduction.ipynb](Introduction.ipynb).  We first describe the physical context of this system and then investigate its characteristic structure and the solution to the Riemann problem.  This system is described in more detail in Chapter 3 of <cite data-cite="fvmhp"><a href="riemann.html#fvmhp">(LeVeque 2002)</a></cite>.

If you wish to examine the Python code for this chapter, please see:

 - [exact_solvers/acoustics.py](exact_solvers/acoustics.py)
 - [exact_solvers/acoustics_demos.py](exact_solvers/acoustics_demos.py)

## Physical setting
The linear acoustic equations describe the propagation of small perturbations in a fluid, such as sound waves.  In [Advection.ipynb](Advection.ipynb) we derived the one-dimensional continuity equation, which describes mass conservation:  
\begin{align} \label{Ac:continuity}
    \rho_t + (\rho u)_x & = 0.
\end{align}  
For more realistic fluid models, we need another equation that determines the velocity $u$.  This typically takes the form of a conservation law for momentum $\rho u$.  Momentum, like density, is transported by fluid motion with corresponding flux $\rho u$.  Additionally, any difference in pressure will also lead to a flux of momentum that is proportional to the pressure difference.  Thus the momentum equation takes the form  
\begin{align} \label{Ac:mom_cons}
(\rho u)_t + (\rho u^2 + P(\rho))_x & = 0,
\end{align}  
where we assumed the system obeys the equation of state $P(\rho)$, where the pressure $P$ is only a function of the density. More general equations of state will be consider, along with fully nonlinear fluid motions in [Euler.ipynb](Euler.ipynb) and [Euler_Tammann.ipynb](Euler_Tammann.ipynb).  Linear acoustics equations focus on the behavior of small perturbations in the system above.

In order to derive acoustics linear equations, note equations (\ref{Ac:continuity}) and (\ref{Ac:mom_cons}) together form a hyperbolic system $q_t+f(q)_x=0$ with  
\begin{align*}
q & = \begin{bmatrix} \rho \\ \rho u \end{bmatrix} & 
f(q) & = \begin{bmatrix} \rho u \\ \rho u^2 + P(\rho) \end{bmatrix}
\end{align*}  

We will further need to make use of the quasilinear form of a given hyperbolic system (which will be used throughout this book):
$$q_t + f'(q) q_x = 0,$$  
where $f'(q)$ denotes the Jacobian of the flux $f$ with respect to the conserved variables $q$.  In the present system, as is often the case, $f$ is most naturally written in terms of so-called primitive variables (in this case $\rho$ and $u$) rather than in terms of the conserved variables $q$.  In order to find the flux Jacobian (and thus the quasilinear form), we first write $f$ in terms of the conserved variables $(q_1,q_2) = (\rho, \rho u)$:  
\begin{align}
f(q) & = \begin{bmatrix} q_2 \\ q_2^2/q_1 + P(q_1) \end{bmatrix}.
\end{align}  

Now we can differentiate to find the flux Jacobian:  
\begin{align*}
f'(q) & = \begin{bmatrix} \partial f_1/\partial q_1 & \partial f_1/\partial q_2 \\
                          \partial f_2/\partial q_1 & \partial f_2/\partial q_2 \end{bmatrix} \\
      & = \begin{bmatrix} 0 & 1 \\ -q_2^2/q_1^2 + P'(q_1) & 2q_2/q_1 \end{bmatrix} \\
      & = \begin{bmatrix} 0 & 1 \\ P'(\rho)-u^2 & 2u \end{bmatrix}.
\end{align*}

Thus small perturbations to an ambient fluid state $\rho_0, u_0$ evolve according to the linearized equations $q_t + f'(q_0) q_x = 0$, or more explicitly
\begin{align*}
\rho_t + (\rho u)_x & = 0 \\
(\rho u)_t + (P'(\rho_0)-u_0^2)\rho_x + 2u_0(\rho u)_x & = 0.
\end{align*}  
As we are only interested in small perturbations of equation (\ref{Ac:mom_cons}), we expand the perturbations $\rho-\rho_0$ and $\rho u - \rho_0 u_0$ as functions of a small parameter $\epsilon$, and then we discard terms of order $\epsilon^2$ and higher. This results in the linear hyperbolic system  
\begin{align*}
p_t + u_0 p_x + P'(\rho_0) u_x & = 0 \\
u_t + \frac{1}{\rho_0} p_x + u_0 u_x & = 0,
\end{align*}
where $p(x,t)$ is the pressure as a function of $x$ and $t$. If the ambient fluid is at rest (i.e. $u_0=0$) and the pressure is directly proportional to the density, then this simplifies to
\begin{align} \label{Ac:main}
 \left[ \begin{array}{c}
p \\
u 
\end{array} \right]_t +  \underbrace{\left[ \begin{array}{cc}
0 & K_0 \\
1/\rho_0 & 0  \\
\end{array} \right]}_{\mathbf{A}}
\left[ \begin{array}{c}
p \\
u \end{array} \right]_x = 0,
\end{align}
where $K_0=\rho_0 P'(\rho_0)$ is referred to as the bulk modulus of compressibility. The system of equations (\ref{Ac:main}) is called the linear acoustics equations.

For the rest of this chapter we work with (\ref{Ac:main}) and let $q=[p,u]^T$.  Then we can write (\ref{Ac:main}) as $q_t + A q_x = 0$.  For simplicity, we also drop the subscripts on $K, \rho$.  Direct calculation reveals that the eigenvectors of $A$ are
\begin{align}
\lambda_1 = -c, \qquad \lambda_2 = c
\end{align}

where $c=\sqrt{{K}/{\rho}}$ will correspond to the speed of sound in a medium with a given density and bulk modulu. The corresponding right eigenvectors are given by
\begin{align*}
r_1 = \begin{bmatrix}\begin{array}{c}-Z\\1\end{array}\end{bmatrix}, \qquad r_2 = \begin{bmatrix}\begin{array}{c}Z\\1\end{array}\end{bmatrix},
\end{align*}

where $Z=\rho c$ is called the acoustic impedance. Defining $R = [r_1 r_2]$ and $\Lambda = diag(\lambda_1, \lambda_2)$, we have $AR = R\Lambda$, or $A = R \Lambda R^{-1}$.  Substituting this into (\ref{Ac:main}) yields
\begin{align*}
q_t + A q_x & = 0 \\
q_t + R \Lambda R^{-1} q_x & = 0 \\
R^{-1}q_t + \Lambda R^{-1} q_x & = 0 \\
w_t + \Lambda w_x & = 0,
\end{align*}
where we have introduced the *characteristic variables* $w=R^{-1}q$.  The last system above is simply a pair of decoupled advection equations for $w_1$ and $w_2$, with velocities $\lambda_1$ and $\lambda_2$; a system we already know how to solve. Thus we see that the eigenvalues of $A$ are the velocities at which information propagates in the solution.

## Solution by characteristics

The discussion above suggests a strategy for solving the Cauchy problem:

1. Decompose the initial data $(p(x,0), u(x,0))$ into characteristic variables $w(x,0)=(w_1^0(x),w_2^0(x,0))$ using the relation $w = R^{-1}q$.
2. Evolve the characteristic variables: $w_p(x,t) = w_p^0(x-\lambda_p t)$.
3. Transform back to the physical variables: $q = Rw$.

The first step in this process amounts to expressing the vector $q$ in the basis given by $r_1, r_2$.  Solving the system $Rw=q$ yields 
\begin{align*}
q = w_1 r_1 + w_2 r_2,
\end{align*} 
where
\begin{align*}
w_1 = \frac{- p + Z u}{2Z}, \ \ \ \ \ \
w_2 = \frac{ p + Z u}{2Z}.
\end{align*}

We visualize this below, where the first plot shows the two eigenvectors, and the second plot shows how $q$ can be expressed as a linear combination of the two eigenvectors, $r_1$ and $r_2$.  In the live notebook you can adjust the left and right states or the material parameters to see how this affects the construction of the Riemann solution.

In [3]:
acoustics_demos.decompose_q_interactive()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTEuMCwgZGVzY3JpcHRpb249dSdwJywgbWF4PTEuMCwgbWluPS0xLjApLCBGbG9hdFNsaWRlcih2YWx1ZT0wLjAsIGRlc2PigKY=


VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXIodmFsdWU9MS4wLCBkZXNjcmlwdGlvbj11J3AnLCBtYXg9MS4wLCBtaW49LTEuMCksIEZsb2F0U2xpZGVyKHZhbHVlPTHigKY=


Output()

In the second and third steps, we evolve the characteristic variables $w$ and then transform back to the original variables. Let's assume the acoustics equations have a bump in pressure with zero velocity as the initial condition. We visualize this below, where the time evolution in the characteristic variables is shown in the first plot, and the time evolution back in one of the original variables (velocity) is shown in the second plot.

In [4]:
acoustics_demos.char_solution_interactive()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSd0JywgbWF4PTEuMiksIEZsb2F0U2xpZGVyKHZhbHVlPTEuMCwgZGVzY3JpcHRpb249dSfigKY=


SEJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXIodmFsdWU9MC4wLCBkZXNjcmlwdGlvbj11J3QnLCBtYXg9MS4yKSwpKSwgVkJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXLigKY=


Output()

Notice how in the characteristic variables $w$ (plotted on the left), each part of the solution simply advects (translates) since each of the characteristics variables simply obeys an uncoupled advection equation. However, back in the original variables the behavior is more complex and cannot be explained simply by advection.

## The Riemann problem

Now that we know how to solve the Cauchy problem, solution of the Riemann problem is merely a special case.  We have the special initial data  
\begin{align*}
q(x,0) = \begin{cases}
q_\ell & \text{if   } x \le 0, \\
q_r & \text{if   } x > 0.
\end{cases}
\end{align*}  
We can proceed as before, by decomposing into characteristic components, advecting, and then transforming back.  But since we know the solution will be constant almost everywhere, it's even simpler to just decompose the jump $\Delta q = q_r - q_\ell$ in terms of the characteristic variables, and advect the two resulting jumps $\Delta w_1$ and $\Delta w_2$:  
\begin{align*}
\Delta q = \Delta w_1 r_1 + \Delta w_2 r_2,
\end{align*}  
Since $R\Delta w = \Delta q$, we have  
\begin{align*}
\Delta w_1 = \frac{-\Delta p + Z\Delta u}{2Z}, \ \ \ \ \ \
\Delta w_2 = \frac{\Delta p + Z\Delta u}{2Z}.
\end{align*}  

Thus the solution has the structure depicted below.

![Fig 4.1: Structure of the Riemann solution for acoustics.](./figures/acoustics_xt_plane.png)

The three constant states are related by the jumps:   
\begin{align}
q_m = q_\ell + \Delta w_1 r_1 = q_r - \Delta w_2 r_2.
\label{eq:acussol}
\end{align}  
Note that the form of the eigenvectors shows that, in any propagating discontinuity, the jump in $p$ is $\pm Z$ times the jump in $u$.  More generally, the eigenvectors of the coefficient matrix of a linear hyperbolic system reveal the relation between jumps in the different components of $q$ across the wave propagating with speed given by its corresponding eigenvalue.  For acoustics, the impedance is the physical parameter that determines this relation.

### A simple solution
Here we provide some very simple initial data, and we call the linear Riemann solver. This will output the three states $q_l$, $q_m$ and $q_r$, and the speeds of the two waves.

In [5]:
# Initial data for Riemann problem
rho = 0.5               # density
bulk = 2.            # bulk modulus
ql = np.array([3,2])   # Left state
qr = np.array([3,-2])  # Right state
# Calculated parameters
c = np.sqrt(bulk/rho)  # calculate sound speed
Z = np.sqrt(bulk*rho)  # calculate impedance
print("With density rho = %g,  bulk modulus K = %g" \
      % (rho,bulk))
print("We compute: sound speed c = %g, impedance Z = %g \n" \
      % (c,Z))

With density rho = 0.5,  bulk modulus K = 2
We compute: sound speed c = 2, impedance Z = 1 



In [6]:
# Call and print Riemann solution
states, speeds, reval = \
    acoustics.exact_riemann_solution(ql ,qr, [rho, bulk])
    
print("The states ql, qm and qr are: ")
print(states, "\n")
print("The left and right wave speeds are:")
print(speeds)

The states ql, qm and qr are: 
(array([[ 3.,  5.,  3.],
       [ 2.,  0., -2.]]), '\n')
The left and right wave speeds are:
[-2.  2.]


As we already showed in the previous section, one way to understand the solution to a linear system like acoustics equations is by looking at the phase plane. We can show the structure of this solution in the $p-u$ phase plane. We show the two initial conditions of the Riemann problem $q_l$ and $q_r$ as points in the phase space; the lines crossing this points correspond to the eigenvectors, $r_1$ and $r_2$. 


The solution of the middle state $q_m$ is simply the intersection of the line generated by $r_1$ passing through $q_l$ and the line generated by $r_2$ passing through $q_r$. The structure of this solution becomes evident from equation (\ref{eq:acussol}). Note there is another intersection between these lines (dashed), these correspond to connecting $q_l$ to $q_m$ with the $r_2$ eigenvector, which is not the correct solution (see equation (\ref{eq:acussol})).

In the live notebook, the cell below allows you to interactively adjust the initial conditions the material parameters as well as the plot range, so that you can explore how the structure of the solution in the phase plane is affected by these quantities.

In [7]:
acoustics_demos.interactive_phase_plane(ql,qr,rho,bulk)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTMuMCwgZGVzY3JpcHRpb249dSckcF9sJCcsIG1heD0xMC4wLCBtaW49MC4wMSksIEZsb2F0U2xpZGVyKHZhbHVlPTIuMCzigKY=


VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShGbG9hdFNsaWRlcih2YWx1ZT0zLjAsIGRlc2NyaXB0aW9uPXUnJHBfbCQnLCBtYXg9MTAuMCwgbWluPTAuMDEpLCDigKY=


Output()

Note that the eigenvectors are given in terms of the impedance $Z$, which depends on the density $\rho$
and the bulk modulus $K$. Therefore, when $\rho$ and $K$ are modified the eigenvectors change and consequently the slope of the lines changes as well.

## Examples
We will use the exact solver in [exact_solvers/acoustics.py](exact_solvers/acoustics.py) and the functions in [exact_solvers/acoustics_demos.py](exact_solvers/acoustics_demos.py) to plot interactive solutions for different interesting examples.

### Shock tube

If there is a jump in pressure and the velocity is $0$ in both initial states (the shock tube problem) then the resulting Riemann solution consists of pressure jumps of equal magnitude propagating in both directions, with equal and opposite jumps in velocity. This is called the shock tube problem since it emulates what would happen inside a shock tube, where the air is static and a separate chamber at the end of the tube is pressurized to a high pressure and then released. This generates a high-pressure wave propagating towards the low-pressure region, while a lower pressure wave is propagating back into the chamber. We will later see that if we use more complex model, like Euler equations, these waves will correspond to a shock and a rarefaction, respectively.   

In [8]:
ql = np.array([5,0])
qr = np.array([1,0])
rho = 1.0
bulk = 4.0
acoustics_demos.riemann_plot_pplane(ql,qr,rho,bulk)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSckdCQnLCBtYXg9MS4wKSwgRHJvcGRvd24oZGVzY3JpcHRpb249dSdDaGFyYWNzLicsIG/igKY=


SEJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXIodmFsdWU9MC4wLCBkZXNjcmlwdGlvbj11JyR0JCcsIG1heD0xLjApLCBEcm9wZG93bihkZXNjcmlwdGlvbj11J0NoYXJhY3MuJywgb3B0aW9ucz3igKY=


Output()

We can also observe the structure of the solution in the phase plane. In the second plot, we show the structure of the solution in the phase plane.

### Reflection from a wall

As another example, suppose the pressure is initially the same in the left and right states, while the velocities are non-zero with $u_r = -u_\ell > 0$.  Particles are converging from both sides and if the initial states have this symmetry, then the result is a middle state $q_m$ in which the velocity is 0 (and the pressure is higher than on either side).

In [9]:
ql = np.array([2,1])  
qr = np.array([2,-1])  
rho = 1.0
bulk = 1.5
acoustics_demos.riemann_plot_pplane(ql,qr,rho,bulk)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSckdCQnLCBtYXg9MS4wKSwgRHJvcGRvd24oZGVzY3JpcHRpb249dSdDaGFyYWNzLicsIG/igKY=


SEJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXIodmFsdWU9MC4wLCBkZXNjcmlwdGlvbj11JyR0JCcsIG1heD0xLjApLCBEcm9wZG93bihkZXNjcmlwdGlvbj11J0NoYXJhY3MuJywgb3B0aW9ucz3igKY=


Output()

We again show the Riemann solution at a given time along with the solution in the phase plane for comparison. Note the symmetry of the solution is also observed in the phase plane solution.

If one discards half the solution for $x>0$ (or for $x<0$), one can view this as a solution to a problem with fluid streaming at a constant velocity toward a solid wall. The result is an acoustic wave that moves away from the wall, and the fluid behind the shock has been decelerated to velocity 0, i.e. it is stationary at the wall.

This type of Riemann solution is critical when imposing solid wall boundary conditions in a numerical method. If ghost cells are introduced outside the domain and the state in the ghost cell set by reflecting the interior solution with the symmetry seen here (equal pressure, negated velocity), then the solution to the Riemann problem at the cell interfaces yields a solution that satisfies the desired boundary conditions. 

Note it is possible to extend the Riemann problem solution for acoustic equations to cases where there are different materials on the left and right side. This is useful to solve the acoustic wave propagation across interfaces or heterogeneous media.

### Interactive phase plane with solution at fixed time

For a more general exploration of the solution to the acoustics equation, we now show an interactive solution of the acoustics equations. The initial condition, $q_l$ and $q_r$, can be modified by dragging and dropping the points in the phase plane plot (in the notebook version, or on [this webpage](http://www.clawpack.org/riemann_book/html/phase_plane_acoustics_small.html)). The solution at some fixed time will be modified accordingly, by showing the three values corresponding values for the pressure and the velocity.  

In [10]:
IFrame(src='html/phase_plane_acoustics_small_notitle.html', 
       width=980, height=340)

### Bump initial condition
In this example, we use the original bump example described in the beginning of this chapter. The initial condition is simply a bump in the pressure, while the velocity equals to zero. Additionally, we set walls at $x=-2$ and $x=2$, so the wave is fully reflected back, and we can see how it interacts with itself. This animation is produced by running a numerical method, and can be viewed in the interactive notebook or on [this webpage](http://www.clawpack.org/riemann_book/html/acoustics_bump_animation.html).

In [11]:
acoustics_demos.bump_animation(numframes = 50)